## Installing requirements and connecting to the database

In [4]:
#Installing requirements
!pip install psycopg2-binary
!pip install psycopg2

In [5]:
#Suppressing warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [6]:
import psycopg2
import json

In [7]:
#Connecting to database
with open('/Users/Tiyas/Desktop/db_cred.json','r') as j:
    cred = json.load(j)

In [8]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    user=cred["user"],
    password=cred["password"])

Connecting to the PostgreSQL database...


In [9]:
cur = conn.cursor()

In [10]:
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

PostgreSQL database version:


In [11]:
# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [ ]:
#Codes to drop tables if jupyter notebook has to be run serially repeatedly
query = """DROP TABLE IF EXISTS authors"""
cur.execute(query)
conn.commit()

In [ ]:
query = """DROP TABLE IF EXISTS publishers CASCADE"""
cur.execute(query)
conn.commit()

In [ ]:
query = """DROP TABLE IF EXISTS books"""
cur.execute(query)
conn.commit()

In [ ]:
query = """DROP TABLE IF EXISTS books_authors"""
cur.execute(query)
conn.commit()

## Reading and cleaning data

In [ ]:
#using dataframes as a transition tool from csv to database
import pandas as pd
df = pd.read_csv('/Users/Tiyas/Desktop/books.csv')

In [ ]:
#we can visualise the numbers of rows and columns in the dataset
df

In [ ]:
#checking for null rows
df2 = df[df.isna().any(axis=1)]

In [ ]:
df2

In [ ]:
#dropping rows with null values since all columns were null
df.dropna(inplace=True)

In [ ]:
#cleaning data for insertion into database
df.rename(columns = {'  num_pages':'num_pages'}, inplace = True)
df['title'] = df['title'].str.replace("\'", "\'\'")
df['publisher'] = df['publisher'].str.replace("\'", "\'\'")
df['authors'] = df['authors'].str.replace("\'", "\'\'")

In [ ]:
#splitting multiple authors for the same book into different rows and creating a new dataframe
df_split_authors = pd.DataFrame(columns=['bookID','title','author','average_rating','isbn','isbn13','language_code','num_pages'
                               ,'ratings_count','text_reviews_count','publication_date','publisher'])
indx = 0
for index, row in df.iterrows():
    authors = row['authors'].split("/")
    for a in authors:
        df_split_authors.loc[indx] = pd.Series({
          'bookID':row['bookID'],
          'title':row['title'],
          'author': str(a),
          'average_rating': row['average_rating'],
          'isbn': row['isbn13'],
          'isbn13': row['isbn'],
          'language_code': row['language_code'],
          'num_pages': row['num_pages'],
          'ratings_count': row['ratings_count'],
          'text_reviews_count': row['text_reviews_count'],
          'publication_date': row['publication_date'],
          'publisher': row['publisher']
        })
        indx = indx + 1

## Code to create three tables - books, authors and publisers and one junction table - books_authors

### Creating authors table

In [ ]:
create_author_table = '''create table authors(
id SERIAL PRIMARY KEY,
name VARCHAR(500),
CONSTRAINT name_unique UNIQUE (name));'''

In [ ]:
cur.execute(create_author_table)
conn.commit()

In [ ]:
authors = list(set(df_split_authors['author']))

In [ ]:
for a in authors:
    query = """INSERT INTO authors (name) VALUES """
    query+= str('(\'' + str(a) + '\')')
    cur.execute(query)
conn.commit()

In [ ]:
query = "select * from authors"
cur.execute(query)
all_authors = pd.read_sql(query,conn)
conn.commit()

In [ ]:
all_authors

### Creating publishers table

In [ ]:
publishers = list(set(df['publisher']))

In [ ]:
create_publisher_table = '''create table publishers(
id SERIAL PRIMARY KEY,
name VARCHAR(1000),
CONSTRAINT p_name_unique UNIQUE (name));'''

In [ ]:
cur.execute(create_publisher_table)
conn.commit()

In [ ]:
for p in publishers:
    query = """INSERT INTO publishers (name) VALUES """
    query+= str('(\'' + str(p) + '\')')
    cur.execute(query)
conn.commit()

In [ ]:
query = "select * from publishers"
cur.execute(query)
all_publishers = pd.read_sql(query,conn)
conn.commit()

In [ ]:
all_publishers

### Creating books table

In [ ]:
create_books_table = """create table books(
    id DECIMAL,
    title VARCHAR(2000),
    average_rating DECIMAL,
    isbn VARCHAR(1000),
    isbn13 VARCHAR(1000),
    language_code VARCHAR(100),
    num_pages DECIMAL,
    ratings_count DECIMAL,
    text_reviews_count DECIMAL,
    publication_date DATE, 
    publisher_id INTEGER,
    CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(id)
);"""

In [ ]:
cur.execute(create_books_table)
conn.commit()

In [ ]:
for index, row in df.iterrows():
    query = """INSERT INTO books (id, title, average_rating, isbn, isbn13,
       language_code, num_pages, ratings_count, text_reviews_count,
       publication_date, publisher_id) values """
    q = "select id from publishers where name=\'" + str(row['publisher']) + "\'"
    cur.execute(q)
    p_id = pd.read_sql(q,conn)
    p_id = p_id['id'].tolist()[0]
    conn.commit()
    query += str('(' + str(row['bookID']) + ',\'' + str(row['title']) + '\',' + 
                 str(row['average_rating']) + ',\'' + str(row['isbn']) + '\',' + str(row['isbn13']) + 
                 ',\'' + str(row['language_code']) + '\',' +  str(row['num_pages']) + ',' + str(row['ratings_count']) + ',' + str(row['text_reviews_count'])
                 + ',\'' + str(row['publication_date']) + '\',' + str(p_id) + ')')
    cur.execute(query)
conn.commit()

In [ ]:
query = "select * from books"
cur.execute(query)
all_books = pd.read_sql(query,conn)
conn.commit()

In [ ]:
all_books

### Creating junction table books_authors

In [ ]:
create_join_table = '''create table books_authors(
bookID DECIMAL,
authorID INTEGER,
PRIMARY KEY (bookID, authorID)
);'''
cur.execute(create_join_table)
conn.commit()

In [ ]:
for index, row in df_split_authors.iterrows():
    q = "select id from authors where name=\'" + str(row['author']) + "\'"
    cur.execute(q)
    a_id = pd.read_sql(q,conn)
    a_id = a_id['id'].tolist()[0]
    conn.commit()
    query = """INSERT INTO books_authors (bookID, authorID) VALUES """
    query+= str('(' + str(row['bookID']) + ',' + str(a_id) + ') ON CONFLICT DO NOTHING')
    cur.execute(query)

In [ ]:
q = "select * from books_authors"
cur.execute(q)
a_id = pd.read_sql(q,conn)
a_id

## Answering SQL Questions

### What are the oldest 5 books in this dataset?

In [ ]:
sql = "select * from books order by publication_date limit 5;"
cur.execute(sql)
oldest_books = pd.read_sql(sql, conn)

In [ ]:
oldest_books

### Which 5 authors have the highest number of ratings?

In [ ]:
sql = "select authors.name, sum(books.ratings_count) as num_ratings from books join books_authors on books.id = books_authors.bookID join authors on books_authors.authorID = authors.id group by authors.name order by num_ratings desc limit 5"
cur.execute(sql)
highest_rated_authors = pd.read_sql(sql, conn)

In [ ]:
highest_rated_authors

### Which 5 language codes have the highest rating average?

In [ ]:
sql = "select language_code, avg(average_rating) as a from books group by language_code order by a desc limit 5"
cur.execute(sql)
language_codes_highest_average = pd.read_sql(sql, conn)

In [ ]:
language_codes_highest_average

In [ ]:
sql = """with temp_table (total_ratings, total_ratings_count, language_code) as 
(select sum(books.ratings_count*books.average_rating) as total_ratings, 
sum(books.ratings_count) as total_ratings_count, books.language_code FROM books 
group by language_code) select language_code, total_ratings/total_ratings_count as total_avg from tempTable 
WHERE total_ratings_count != 0 order by total_avg desc limit 5;
"""
cur.execute(sql)
language_codes_highest_average = pd.read_sql(sql, conn)

In [ ]:
language_codes_highest_average

### Which publishers publish in the highest number of different languages?

In [ ]:
sql = "select publishers.name, count(distinct(books.language_code)) as num_languages from books join publishers on books.publisher_id = publishers.id group by publishers.name order by num_languages desc limit 1;"
cur.execute(sql)
publisher_with_highest_languages = pd.read_sql(sql, conn)

In [ ]:
publisher_with_highest_languages